In [ ]:
#批量归一层
import d2lzh as d2l
from mxnet import autograd,gluon,init,nd
from mxnet.gluon import nn
def batch_norm(X,gamma,beta,moving_mean,moving_var,eps,momentum):
    #autograd判断当前模式为训练模式还是测试模式
    if not autograd.is_training():
        # 如果是在预测模式下，直接使⽤传⼊的移动平均所得的均值和⽅差
        X_hat = (X - moving_mean) / nd.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2,4)
        if len(X.shape) == 2:
            # 使⽤全连接层的情况，计算特征维上的均值和⽅差
            mean = X.mean(axis=0)
            var = ((X - mean) ** 2).mean(axis=0)
        else:
            # 使⽤⼆维卷积层的情况，计算通道维上（axis=1）的均值和⽅差。这⾥我们需要保持
            # X的形状以便后⾯可以做⼴播运算
            mean = X.mean(axis=(0, 2, 3), keepdims=True)
            var = ((X - mean) ** 2).mean(axis=(0, 2, 3), keepdims=True)
        # 训练模式下⽤当前的均值和⽅差做标准化
        X_hat = (X - mean) / nd.sqrt(var + eps)
        # 更新移动平均的均值和⽅差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var    
    Y = gamma * X_hat + beta # 拉伸和偏移
    return Y, moving_mean, moving_var

